<a href="https://colab.research.google.com/github/AdhithJCB/bdmc-demo/blob/main/notebooks/01_biomolecular_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biomolecular Visualization Demo (PDB + Python)

**Goal:** Load a protein structure from the PDB and use Python to compute/visualize simple structural features.

**Run cells top → bottom.**  
If anything breaks: **Runtime → Restart session → Run all**

You'll learn:
- how to download a structure (PDB)
- how to extract Cα (CA) coordinates
- how to compute distances to a chosen residue
- how to visualize the structure in 3D


In [ ]:
!pip -q install biopython py3Dmol

## Step 1: Download a protein structure (PDB)

We’ll download a structure from the **RCSB Protein Data Bank**.

**Try it:** change `pdb_id` to another ID (e.g., `4hhb`, `1crn`, `7tim`) and rerun.


In [ ]:
import urllib.request
import os

#=== Edit this ===
pdb_id = "1ubq"   # 4-character PDB ID (letters/numbers)

url = f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb"
out = f"{pdb_id.lower()}.pdb"

urllib.request.urlretrieve(url, out)

print("Downloaded:", out)
assert os.path.exists(f"{pdb_id}.pdb"), "PDB file not found: re-run the download cell."
print("File exists?", os.path.exists(out))
print("File size:", os.path.getsize(out), "bytes")


## Step 2: Parse the PDB and extract coordinates

We'll use **BioPython** to parse the structure.
To keep things simple, we’ll focus on:
- the **first model**
- a selected **chain**
- the **Cα (CA)** atom for each standard amino acid residue

Why Cα? It's a common “backbone” representative for geometry calculations.


In [ ]:
from Bio.PDB import PDBParser
import numpy as np

def load_ca_trace(pdb_path, preferred_chain=None):
    """
    Returns:
      chain_id: chain identifier (e.g., 'A')
      coords: (N, 3) CA coordinates
      res_ids: list of residue numbers corresponding to coords
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("struct", pdb_path)

    model = next(structure.get_models())  #first model
    chains = list(model.get_chains())

    if preferred_chain is not None:
        chain = next((c for c in chains if c.id == preferred_chain), None)
        if chain is None:
            print(f"Preferred chain '{preferred_chain}' not found; using first chain instead.")
            chain = chains[0]
    else:
        chain = chains[0]

    residues = []
    coords = []
    res_ids = []

    for r in chain.get_residues():
        # r.id[0] == " " means standard residue (not water/hetero)
        if r.id[0] == " " and "CA" in r:
            residues.append(r)
            coords.append(r["CA"].get_coord())
            res_ids.append(r.id[1])  # residue number

    coords = np.array(coords, dtype=float)
    return chain.id, coords, res_ids

#Optional: set preferred_chain = 'A' for multi-chain proteins
preferred_chain = None

chain_id, coords, res_ids = load_ca_trace(out, preferred_chain=preferred_chain)

print(f"Using chain: {chain_id}")
print(f"CA atoms found: {len(coords)}")
assert len(coords) > 0, "No CA atoms found: try a different PDB ID."
print(f"Residue number range: {min(res_ids)} to {max(res_ids)}")


## Step 3: Distances from a chosen residue

Pick a residue **index** `i` (0 to N-1).  
We compute distances from its Cα to all other Cα atoms.

Note: `i` is an *index into our CA list*, while `res_ids[i]` is the actual **PDB residue number**.


In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

@interact(i=IntSlider(min=0, max=len(coords)-1, step=1, value=10, description="i"))
def show_distances(i=10):
    ref = coords[i]
    dists = np.linalg.norm(coords - ref, axis=1)

    plt.figure()
    plt.hist(dists, bins=20)
    plt.title(f"Distances from residue {res_ids[i]} (CA) in chain {chain_id}")
    plt.xlabel("Distance (Å)")
    plt.ylabel("Count")
    plt.show()

    order = np.argsort(dists)
    closest = [(res_ids[j], float(dists[j])) for j in order[1:11]]  #skip itself at order[0]
    print("10 closest residues (resnum, Å):")
    print(closest)

## Step 4: 3D visualization (py3Dmol)

We’ll visualize the structure directly from the PDB ID and highlight **one residue**.


In [ ]:
import py3Dmol

#=== Edit this ===
i = 55  #index into our CA list (same meaning as the slider above)

sel_resnum = res_ids[i]

view = py3Dmol.view(query=f"pdb:{pdb_id.lower()}")
view.setStyle({'cartoon': {'color': 'spectrum'}})

#Highlight selected residue
view.addStyle({'chain': chain_id, 'resi': str(sel_resnum)}, {'stick': {'color': 'red'}})

view.zoomTo({'chain': chain_id, 'resi': str(sel_resnum)})
view.show()

print("Highlighted residue number:", sel_resnum, "in chain", chain_id)


## Mini-challenge A: Find neighbors within 8Å

Pick a distance threshold and find residues within that radius (excluding itself).


In [ ]:
threshold = 8.0

#reassign i here (again index into CA list)
i = 23

ref = coords[i]
dists = np.linalg.norm(coords - ref, axis=1)

neighbors = [res_ids[j] for j in range(len(dists)) if (dists[j] < threshold and j != i)]
print(f"Residues within {threshold}Å of residue {res_ids[i]}:", neighbors[:25])
print("Count:", len(neighbors))


## Mini-challenge B: Highlight neighbors in 3D

We’ll highlight:
- selected residue in **red**
- neighbors within threshold in **yellow**


In [ ]:
import py3Dmol
import numpy as np

threshold = 8.0
i = 10

ref = coords[i]
dists = np.linalg.norm(coords - ref, axis=1)
neighbor_resnums = [res_ids[j] for j in range(len(dists)) if (dists[j] < threshold and j != i)]

view = py3Dmol.view(query=f"pdb:{pdb_id.lower()}")
view.setStyle({'cartoon': {'color': 'spectrum'}})

#neighbors
for rnum in neighbor_resnums:
    view.addStyle({'chain': chain_id, 'resi': str(rnum)}, {'stick': {'color': 'yellow'}})

#selected residue
view.addStyle({'chain': chain_id, 'resi': str(res_ids[i])}, {'stick': {'color': 'red'}})

view.zoomTo({'chain': chain_id, 'resi': str(res_ids[i])})
view.show()

print("Selected:", res_ids[i])
print("Neighbors highlighted:", len(neighbor_resnums))
